In [ ]:
import sys
sys.path.append('../../')

from os import listdir

from hdf5storage import loadmat, savemat 
import numpy as np 
from matplotlib import pyplot as plt

from scipy import signal 
from hnlpy.timeop import makefiltersos
from hnlpy.timeop import narrowfilter 
from scipy.signal import sosfiltfilt
from scipy.signal import hilbert
from scipy.signal import savgol_filter
from scipy.fftpack import fft
from scipy import stats # for zscoring

from sklearn.covariance import GraphicalLassoCV

from numpy.polynomial.polynomial import polyfit, polyval

import matplotlib.pyplot as plt

In [ ]:
# load data
def loaddata(filename):
    data = loadmat(filename)
    bpchan = int(data['bpchan'][0][0])
    channels = data['channels'][0]
    conditionNames = data['conditionNames'][0]
    conditions = data['conditions'][0]
    eeg = dict()
    eeg[0] = data['dataL'][0]
    eeg[1] = data['dataR'][0]
    intervals = data['intervals'][0]
    labels = data['labels'][0]
    samples = data['samples'][0]
    session = int(data['session'][0])
    sessionTypes = data['sessionTypes'][0]
    sr = int(data['sr'][0])
    #Labels and condition names doesnt come over cleanly from matlab 
    chan_labels = list()
    condition_Names = list()
    #pythonify condition index
    condition_index = list()
    for j in range(len(labels)):
        chan_labels.append(labels[j][0])
    for j in range(len(conditionNames)):
        condition_Names.append(conditionNames[j][0])
    for j in range(12):
        condition_index.append(int(conditions[j]-1))
    return eeg, intervals, samples, condition_index, session, sr, bpchan, condition_Names, channels, chan_labels, sessionTypes   


In [ ]:
def avref(datax):
	ref = np.mean(datax,axis = 1)
	refmat = np.tile(ref,(np.shape(datax)[1],1))
	datax = datax - np.transpose(refmat)
	return datax

In [ ]:
# build the narrow band filters
srnew = 200 
downsample = 10 
# passbands = [[1,3],[3.5,6.5],[7,10],[10.5,13.5],[14 ,20],[21,29],[30,49.5]]
# stopbands = [[0.75,3.25],[3.25,6.75],[6.75,10.25],[10.25,13.75],[13.75,20.5],[20.5,29.5],[29.5,50]]
# bandlabels = ['Delta','Theta','Alpha','Mu','Beta1','Beta2','Gamma']
passbands = [[3.5,6.5],[7,10],[10.5,13.5],[14 ,20],[21,29]]
stopbands = [[3.25,6.75],[6.75,10.25],[10.25,13.75],[13.75,20.5],[20.5,29.5]]
bandlabels = ['Theta','Alpha','Mu','Beta1','Beta2']
esos = dict()
for j in range(len(passbands)):
    esos[j],w,h = narrowfilter(passbands[j],stopbands[j],samplingrate = srnew)
    plt.plot(w[1:500],20*np.log10(np.abs(h[1:500])))

plt.title('band pass filters')
plt.xlabel('frequency (Hz)')

In [ ]:
# make a low pass filter @ 50 Hz 
sr=2000
sos_low, w,h = makefiltersos(sr,50,55)

In [ ]:
ncv = 5
pc_lasso = GraphicalLassoCV(cv=ncv, n_jobs=-10)

In [ ]:
# organize file in time sequence
filedir="../../"
pathname = 'Cleaned_data/'
filedates=[20220713,20220721,20220804,20220808,20220810,20220811,20220815,20220816,20221003,2022100401,2022100402,20221005]
numSes=len(filedates)

In [ ]:
# This will take a long time, you don't want to run it again
# script here just as a record
chan_prec_all=np.zeros((12,2,12,5,64,64))
for ses in range(12):
	filename='clean_'+str(filedates[ses])+'.mat'
	[eeg, intervals, samples, condition_index, session, sr, bpchan, 
				condition_Names, channels, chan_labels, sessionTypes] \
					= loaddata(filedir+pathname+filename)
	for trial in range(12):
		nsamp1 = np.shape(eeg[0][trial])[0]
		nsamp2 = np.shape(eeg[1][trial])[0]
		nsamp = np.min((nsamp1,nsamp2))
		for subj in range(2): 
			trialdata = avref(eeg[subj][trial][0:nsamp,0:32]) 
			trialdata = sosfiltfilt(sos_low,trialdata,axis=0)
			trialdatanew = trialdata[range(0,nsamp,downsample),:]
			for frequency in range(5):
				filtdata = sosfiltfilt(esos[frequency],trialdatanew,axis = 0,padtype ='odd')
				hilbertdata = hilbert(filtdata,axis = 0)
				# Combine real and imaginary parts
				chanDataReal = np.concatenate((np.real(hilbertdata), np.imag(hilbertdata)), axis=1)
				# Normalize data
				chanDataReal = chanDataReal * (1 / np.mean(np.abs(chanDataReal)))
				pc_lasso.fit(chanDataReal)
				precision = pc_lasso.precision_
				y = np.diag(precision)
				yy = np.outer(y,y)
				yy = np.sqrt(yy)
				chan_prec_all[ses,subj,trial,frequency,:,:] = precision/yy
# 12s x 5 x 144 /3600 = 5 h

In [ ]:
outdict = dict()
outdict['chan_prec_all'] = chan_prec_all
savemat("chan_prec_all.mat",outdict,store_python_metadata = True) 